### For installing dlib 
pip install https://pypi.python.org/packages/da/06/bd3e241c4eb0a662914b3b4875fc52dd176a9db0d4a2c915ac2ad8800e9e/dlib-19.7.0-cp36-cp36m-win_amd64.whl#md5=b7330a5b2d46420343fbed5df69e6a3f

## Haar cascade

This is one of the oldest but still a relevant object detection method which is super simple but very effective. <br>
The logic behind is simple.....If we want to detect a face: Find the boxes where all the facial features are present.<br>
But how to find the facial features<br>
Thats where haar-like features come in....which are:<br>
<img src = "Notes_image/haar_features.jpg">
<br>
These shapes can be used to estimate the features of a face (or any object for that matter) and the goal is to find a box which has all of them.<br>
<img src = "Notes_image/face_withhaar.jpg">
<br>
Traning here is very exhaustive though. Since we are not defining the location of the object, the algorithm need to determine that on its own. This makes tranining a verrry long process..... 
<br><br><b>The steps for training are:</b>
- Take some (actually a lot) positive and some (again...not exactly "some") negative images.
- Convert all the images to grayscale.
- Scale down the positive images to 32x32.
- Now feed that to a training program and wait for an eternity (or get a better pc if you don't want to).....

In [15]:
#Import the cascades....
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')
smile_cascade = cv2.CascadeClassifier('haarcascade_smile.xml')

In [25]:
video_capture.release()
cv2.destroyAllWindows()

In [30]:
face

array([[181, 210, 222, 222]], dtype=int32)

In [31]:
#Displaying the image
video_capture = cv2.VideoCapture(0)
while True:
    #Getting the frames and the grayscale
    _, frame = video_capture.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    #Get the co-ordinates for the face
    face = face_cascade.detectMultiScale(gray, 1.3, 5) #the first one is the image, second is the scale and the last is the neighbours
    #Face is a list of x,y,w,h of all the faces that were detected
    for (x,y,w,h) in face:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)

        #2 regions of interest for the detection and the drawing....
        roi_gray = gray[y:y+h,x:x+w]
        roi_color = frame[y:y+h,x:x+w]

        #Get the co-ordinates for the eyes
        eyes = eye_cascade.detectMultiScale(roi_gray,1.5,3)
        for (x1,y1,w1,h1) in eyes:
            #Draw a rectangle on the eye
            cv2.rectangle(roi_color,(x1,y1),(x1+w1,y1+h1),(0,255,0),2)

        #Get the co-ordinates for that smile
        smiles = smile_cascade.detectMultiScale(roi_gray,1.8,25)
        for (x2,y2,w2,h2) in smiles:
            #Now mark that smile....
            cv2.rectangle(roi_color,(x2,y2),(x2+w2,y2+h2),(0,0,255),2)

    #Now, lets look at the results.....
    cv2.imshow('Video', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
video_capture.release()
cv2.destroyAllWindows()

## The Final Project: Drowsiness detector using DLIB

In [32]:
#Importing the libraries first
from scipy.spatial import distance
import imutils
import dlib
import cv2

In [ ]:
#Lets make the detection and prediction objects
detect = dlib.get_frontal_face_detector()
predict = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

### Lets see how the detection works...
No need to code this one out though......

In [35]:
from imutils import face_utils
import dlib
import cv2

cap = cv2.VideoCapture(0)
 
while True:
    # Getting out image by webcam 
    _, image = cap.read()
    # Converting the image to gray scale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
    # Get faces into webcam's image
    rects = detector(gray, 0)
    
    # For each detected face, find the landmark.
    for (i, rect) in enumerate(rects):
        # Make the prediction and transfom it to numpy array
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)
    
        # Draw on our image, all the finded cordinate points (x,y) 
        for (x, y) in shape:
            cv2.circle(image, (x, y), 2, (0, 255, 0), -1)
    
    # Show the image
    cv2.imshow("Output", image)
    
    k = cv2.waitKey(5) & 0xFF
    if k == 27:
        break

cv2.destroyAllWindows()
cap.release()

### Now for the goal, we will alert the driver if thier eyes are closed.
Now, using dlib we can get 6 points on each eye. Lets use them.....
<br>
For the first step, we'll need the aspect ratio of the eye which will tell us whether the eyes are closed or not:
<img src = "Notes_image/eye3.jpg">
If we find the eyes closed, we can alert the driver. The closed eyes can be determined by a threshold. Lets keep it 0.25. But there is a catch...<br>
Even when blinking, people close their eyes. So, we need to wait for some frame before alerting the driver. So lets keep that number as 10.
<br>
With these in mind, lets begin....

In [34]:
#Copy this fuction....it returns the acpect ration
def eye_aspect_ratio(eye):
    
    A = distance.euclidean(eye[1], eye[5])
    B = distance.euclidean(eye[2], eye[4])
    C = distance.euclidean(eye[0], eye[3])
    
    ear = (A + B)/(2.0 * C)
    return ear

In [38]:
#Now, lets get the index of those eyes.....
(lstart, lend) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rstart, rend) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

In [37]:
#Now for the frames and the aspect ration threshold.....
frames_pass = 10
thresh = 0.25
#Just like we decided.......

In [ ]:
cam = cv2.VideoCapture(0)
#To keep the track of the frames.....
flag = 0
while True:
    #Record the video...
    _, frame = cam.read()
    
    #Now, the usual stuff
    frame = cv2.flip(frame, 1)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    #Find the face in the grayscale image......
    faces = detect(gray)
    
    for face in faces:
        #Getting the features form the face
        face = predict(gray, face)
        face = face_utils.shape_to_np(face)
        
        #Getting the points on for the left and the right eyes
        leftEye = face[lstart:lend]
        rightEye = face[rstart:rend]
        
        #Calculating the aspect ration for both the eyes
        leftEar = eye_aspect_ratio(leftEye)
        rightEar = eye_aspect_ratio(rightEye)
        
        #Finding the net aspect ration.....
        ear = (leftEar + rightEar)/2.0
        
        #Lets draw the convex hull for the sake of it.....it'll make it look cool....
        left_conv = cv2.convexHull(leftEye)
        right_conv = cv2.convexHull(rightEye)
        cv2.drawContours(frame, [left_conv], -1, (0, 255, 0), 1)
        cv2.drawContours(frame, [right_conv], -1, (0, 255, 0), 1)
        
        #Now for the drowsyness detector part.....
        if ear < thresh:
            flag += 1
            
            if flag > frames_pass:
                #Alert the user if he is sleepy....
                
                cv2.putText(frame, "********ALERT*********",
                           (10, 30), cv2.FONT_HERSHEY_COMPLEX, 0.7, (0, 0, 255), 2)
                cv2.putText(frame, "YOU ARE DROWSY",
                           (10, 324), cv2.FONT_HERSHEY_COMPLEX, 0.7, (0, 0, 255), 2)
        
            else:
                cv2.putText(frame, "AWAKE!!!",
                           (10, 324), cv2.FONT_HERSHEY_COMPLEX, 0.7, (0, 0, 255), 2)

        
        else:
            #Reset the counted number of frames
            flag = 0 
    
    cv2.imshow("Live", frame)
   
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
cam.release()
cv2.destroyAllWindows()